# Required libraries

In [1]:
import cv2
import time
import os
import json
import itertools

# Define auxiliary functions

In [2]:
def reduce_timer(cur, prev, timer):
    if ( cur - prev ) >= 1:
        prev = cur
        timer -= 1
    
    return prev, timer

# Displays a countdown for "timer" seconds before recording starts
def countdown(timer):
    prev = time.time()
    while timer >= 0:
        ret, frame = cap.read()
        show_message(frame, timer)
        cv2.imshow('', frame)
        cv2.waitKey(1)
 
        prev, timer = reduce_timer(time.time(), prev, timer)
    cv2.destroyAllWindows()

# Records the video feed for the specified "timer" seconds
def record(out, timer):
    num_frames = int(RECORD_TIME * FPS)
#     while timer >= 0:
    for _ in range(num_frames):
        ret, frame = cap.read()
        cv2.imshow('', frame)
        out.write(frame)
        cv2.waitKey(1)
                
#         prev, timer = reduce_timer(time.time(), prev, timer)
    cv2.destroyAllWindows()
    out.release()

# Draws text on video
def show_message(frame, message, coords=(250, 250), font_size=7):
    cv2.putText(frame, str(message), coords, cv2.FONT_HERSHEY_SIMPLEX, font_size, (0, 255, 255), 4, cv2.LINE_AA)

# def get_first_label(indexes):
#     current_index = None
    
#     for i, index in enumerate(indexes):
#         if current_index is None:
#             current_index = index
#         else:
#             if indexes[current_index] != indexes[index]:
#                 return i
#             else:
#                 current_index = index
#     return 0

def get_next_label_index(values):
    return values.index(min(values))
    
def label_generator(labels, indexes):
    num = get_next_label_index(tuple(indexes.values()))
    
    while True:
        yield num + 1, labels[num]
#         num = ( num + 1 ) % len(labels)
        num = get_next_label_index(tuple(indexes.values()))

# Define constants

In [3]:
ROOT_FOLDER = 'dataset_multi_signer'
# CAMBIAR AQUI
SIGNER = 'signer1'
labels = (
            'good_morning',
            'good_afternoon',
            'good_evening',
            'hi',
            'bye',
            'see_you_tomorrow',
            'see_you_later',
            'family',
            'mom',
            'dad',
            'son',
            'brother',
            'cousin',
            'person',
            'neighbour',
            'kid',
            'youngster',
            'adult',
            'elder',
            'baby'
        )
partitions = ('train', 'test')
label_count_txt = 'label_count.txt'
ALL_DIR = 'all'
RECORD_TIME = 1.5
WAIT_TIME = 2
FPS = 30

# Create folders

In [4]:
if ROOT_FOLDER in os.getcwd():
    os.chdir('..')

if not os.path.exists(ROOT_FOLDER):
    os.makedirs(ROOT_FOLDER)

os.chdir(ROOT_FOLDER)

if not os.path.exists(ALL_DIR):
    os.makedirs(ALL_DIR)

# Create "all" folder for storing samples by label
if os.path.exists(ALL_DIR):
    for label in labels:
        path = os.path.join(ALL_DIR, label)
        if not os.path.exists(path):
            os.makedirs(path)
            print(f'Folder {path} created')

    # Create partition/label folders
    for partition, label in itertools.product(partitions, labels):
        path = os.path.join(partition, label)
        if not os.path.exists(path):
            os.makedirs(path)
            print(f'Folder {path} created')

# Load and check indexes for each label

In [5]:
#Create file and add default 0 count dictionary
if not os.path.exists(label_count_txt):
    with open(label_count_txt, 'w+') as f:
        default_label_count = {label: 0 for label in labels}
        f.write(json.dumps(default_label_count))
        print(f'File {label_count_txt} with default label count dictionary created')

# Verify the label count and number of files in folder match
# for label in labels:
#     if not indexes[label] == len([name for name in files if os.path.isfile(os.path.join(ALL_DIR, name)) and name.startswith(label)]):
#         raise Exception(f'{label} - Label count differs from actual number of files in directory')

# Record video

In [12]:
timer = WAIT_TIME
record_timer = RECORD_TIME
cap = cv2.VideoCapture(0)

with open(label_count_txt, 'r') as f:
    label_count = f.read()

if not label_count:
    raise Exception(f'File {label_count_txt} is empty')
    
indexes = json.loads(label_count)
gen = label_generator(labels, indexes)
label_index, label = next(gen)

if (cap.isOpened() == False):
    raise Exception("Camera is not open")

cap.set(3, 640)
cap.set(4, 480)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
    
while cap.isOpened():
    ret, frame = cap.read()
    
    if ret == False:
        break
    
    show_message(frame, f'{label}-{label_index}', (0, int(0.95 * frame_height)), 1)
    cv2.imshow('', frame)
        
    if cv2.waitKey(1) & 0xFF == ord('c'):
        cv2.destroyAllWindows()
        
        countdown(timer)

        indexes[label] += 1
        out = cv2.VideoWriter(f'{label}-{SIGNER}-{indexes[label]}.mp4',
                              cv2.VideoWriter_fourcc('D','I','V','X'),
                              FPS, (frame_width, frame_height))
        
        record(out, record_timer)
        
        timer = WAIT_TIME
        record_timer = RECORD_TIME
        label_index, label = next(gen)
        
        with open(label_count_txt, 'w') as f:
            f.write(json.dumps(indexes))
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

640.0
480.0
